판다스의 그룹연산은 데이터를 집계하거나, 변환하는 등의 작업을 한 번에 처리할 수 있는 강력한 기능  
분할 - 반영 - 결합 하는 과정을 거치게 됨  
분할 : 어떠한 기준으로 데이터를 나누는 행위  
반영 : 함수 등을 적용하여 데이터를 처리하는 것  
결합 : 처리한 결과를 다시 합치는 것  

SQL의 GROUPBY 구문과 비슷한데 분할-반영-결합은 오래 전부터 빅데이터를 처리하는데 사용했던 방법들임 

# groupby 메서드로 평균값 구하기

수집한 데이터를 바탕으로 평균, 합등을 계산하여 의미있는 값을 도출해 내는 것을 '집계'라고함  
데이터를 집계하면 전체 데이터를 요약, 정리하여 볼 수 있기에 데이터 분석이 훨씬 편해짐.  
갭마인더 데이터 셋을 사용 


In [1]:
import pandas as pd 
df = pd.read_csv('data/gapminder.tsv', sep='\t')

year 열을 기준으로 데이터를 그룹화 한 후, lifeExp 열의 평균을 구함

In [2]:
avg_lifeExp_by_year = df.groupby('year')['lifeExp'].mean()
avg_lifeExp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

# 분할-반영-결합 과정 살펴보기
위의 groupby의 연산 과정을 자세히 하나씩 살펴봄

groupby 에 'year'열을 전달하면, **연도별로 데이터를 나누는 과정**을 수행함.  
열이름을 전달하면 **분할** 과정이 먼저 일어나게 됨.  

In [3]:
df.year.unique()

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007])

그런 다음 연도별로 데이터를 추출한 다음, 평균값을 구하는 과정이 **반영**과정임.  
예시로 1952년도를 기준으로 과정을 설명

In [4]:
y1952 = df.loc[df['year']==1952, :]
y1957 = df.loc[df['year']==1957, :]
y1962 = df.loc[df['year']==1962, :]
y1967 = df.loc[df['year']==1967, :]
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


아래 과정을 1952년 뿐만 아니라 모든 연도에 대해서 시행

In [5]:
y1952_mean = y1952['lifeExp'].mean()
print(y1952_mean)

49.05761971830987


In [6]:
y1957_mean = y1957['lifeExp'].mean()
print(y1957_mean)

51.50740112676054


In [7]:
y1962_mean = y1962['lifeExp'].mean()
print(y1962_mean)

53.60924901408449


In [8]:
y1967_mean = y1967['lifeExp'].mean()
print(y1967_mean)

55.67828957746479


마지막으로 연도별로 계산된 lifeExp 의 평균 값을 합치게 되면 **결합** 과정이 완료됨

In [9]:
df2 = pd.DataFrame({'year':[1952, 1957, 1962, 2967],
                   '':[y1952_mean, y1957_mean, y1962_mean, y1967_mean]})
df2

,year,
0,1952,49.057620
1,1957,51.507401
2,1962,53.609249
3,2967,55.678290


### groupby 메서드와 함께 사용하는 집계 메서드
|메서드|설명|
|:---:|:---:|
|count|누락값을 제외한 데이터 수를 반환|
|size|누락값을 포함한 데이터 수를 반환|
|**mean**|**평균값 반환**|
|std|표준편차 반환|
|**min**|**최솟값 반환**|
|quantile(q=0.25)|백분위수 25%|
|quantile(q=0.50)|백분위수 50%|
|quantile(q=0.75)|백분위수 75%|
|**max**|**최댓값 반환**|
|sum|전체 합 반환|
|var|분산 반환|
|sem|평균의 표준편차 반환|
|**describe**|**데이터 수, 평균, 표준편차, 백분위수(25%, 50%, 75%), 최댓값을 모두 반환**|
|first|첫번째 행 반환|
|last|마지막 행 반환|
|nth|n 번째 행 반환|

# 평균값을 구하는 사용자 함수와 groupby 메서드

라이브러리에서 제공하는 집계 메서드가 원하는 계산을 지원하지 않는 경우 직접 함수를 만들어서 groupby 와 함께 사용해야 함.   
사용자 함수와 groupby 메서드를 조합하려면, agg 메서드를 이용해야 함.

> **DataFrame.agg(func=None, axis=0, *args, *kwargs)**  
Aggregate(집합, ~을 모으다) using one or more operations over the specified axis.



In [10]:
# 입력받은 열의 평균 값을 구하는 함수 
def my_mean(values):
    n = len(values)
    sum = 0
    for item in values:
        sum += item
        
    return sum / n

In [11]:
# 사용자 함수와 groupby 메서드 조합, agg
agg_my_mean = df.groupby('year')['lifeExp'].agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

2개의 인잣값을 받아 처리하는 사용자 함수를 만들어 조합  
첫번째 인자와 두번째 인자와의 평균의 차이를 구해주는 함수  

In [12]:
# 평균의 차이를 구해주는 함수
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
        
    mean = sum / n
    return mean - diff_value

In [13]:
# 전체 평균 수명 
global_mean = df['lifeExp'].mean()
print(global_mean)

# 연도별 수명의 기대수명 평균과, 전체 기대수명 평균과의 차이
agg_mean_diff = df.groupby('year')['lifeExp'].agg(my_mean_diff, diff_value=global_mean)
print(agg_mean_diff)

59.47443936619713
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


# 집계 메서드를 리스트, 딕셔너리에 담아 전달하기
여러 집계 메서드를 한 번에 사용하고 싶다면, 집계 메서드를 리스트 or 딕셔너리에 담아 agg 메서드에 전달

In [14]:
# lifeExp 열의 '0이 아닌 값의 개수', '평균', '표준편차' 를 한번에 계산 
import numpy as np

gdf = df.groupby('year')['lifeExp'].agg([np.count_nonzero, np.mean, np.std])
print(gdf)


      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


딕셔너리의 **키**로 집계 메서드를 적용할 **열 이름을** 전달하고, **값**으로 **집계 메서드**를 전달

In [15]:
gdf_dict = df.groupby('year').agg({'lifeExp':'mean', 'pop':'median', 'gdpPercap':'median'})
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371109


# 표준 점수 계산하기
데이터 변환 메서드에 대해서 알아봄.  
데이터 변환 메서드는 데이터와 메서드를 1:1로 대응시켜 계산하기 때문에 데이터 양이 줄어들지 않음  

통계분야에서는 데이터의 평균과 표준편차의 차이를 표준 점수라고 부름  
표준점수를 구하면 변환한 데이터의 평균 값이 0이 되고, 표준편차는 1이 됨 (0~1 사이의 값으로 오게됨)  
그러면, 데이터가 표준화 되어 서로 다른 데이터를 비교하기 쉬워짐  

In [16]:
# 표준점수 계산하는 함수
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [17]:
# 각 연도별 lifeExp 열의 표준점수를 계산한 것
transform_z = df.groupby('year')['lifeExp'].transform(my_zscore)
print(transform_z)

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
          ...   
1699   -0.081621
1700   -0.336974
1701   -1.574962
1702   -2.093346
1703   -1.948180
Name: lifeExp, Length: 1704, dtype: float64


my_zscore 함수는 데이터를 표준화 할 뿐 집계는 하지 않음, 데이터의 양이 줄지 않음.  

In [18]:
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


# 누락값을 평균값으로 처리하기
이미 6장에서 누락값을 처리하는 방법에 대해 알아봄.  
하지만 가끔 누락값을 평균값으로 처리하는 것이 좋은 방법일 때가 있음 

In [19]:
# 리스트나 배열의 요소를 랜덤하게 섞어줌
x = np.random.permutation([1, 2, 3, 4, 5])
print(x)

[1 5 4 3 2]


In [20]:
import seaborn as sns 
import numpy as np

np.random.seed(42)

# tips 데이터셋을 10개만 불러와, total_bill 열의 임의의 값을 NaN으로 바꿈
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,NaN,2.00,Male,No,Sun,Dinner,4
211,NaN,5.16,Male,Yes,Sat,Dinner,4
198,NaN,2.00,Female,Yes,Thur,Lunch,2
176,NaN,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


그런데 단순히 total_bill의 평균 값으로 누락값들을 채워주면 안됨.  
현재 데이터는 sex열을 보았을때 Female 보다 male의 수가 더 많음.  
그러므로 sex 를 구분해서 평균값을 따로 적용해야함. 그렇지 않으면 데이터가 male에 편향된 값을 가질 수 있음  
성별로 그룹화 -> 성별별 평균 적용 

In [21]:
count_sex = tips_10.groupby('sex')[['total_bill', 'tip']].count()
count_sex

,total_bill,tip
sex,,
Male,4,7
Female,2,3


In [22]:
# total_bill 열의 데이터를 받아 평균값을 구하는 함수 
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [23]:
total_bill_group_mean = tips_10.groupby('sex')['total_bill'].transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
24,19.82,3.18,Male,No,Sat,Dinner,2,19.8200
6,8.77,2.00,Male,No,Sun,Dinner,2,8.7700
153,NaN,2.00,Male,No,Sun,Dinner,4,17.9525
211,NaN,5.16,Male,Yes,Sat,Dinner,4,17.9525
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.9300
176,NaN,2.00,Male,Yes,Sun,Dinner,2,17.9525
192,28.44,2.56,Male,Yes,Thur,Lunch,2,28.4400
124,12.48,2.52,Female,No,Thur,Lunch,2,12.4800
9,14.78,3.23,Male,No,Sun,Dinner,2,14.7800
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.3800


# 데이터 필터링 사용하기 ─ filter 메서드

그룹화한 데이터에서 원하는 데이터를 걸러내고 싶다면 데이터 필터링을 사용하면 됨.  
데이터 필터링을 사용하면 기준에 맞는 데이터를 걸러낼 수 있음

In [24]:
tips = sns.load_dataset('tips')
print(tips.shape)

(244, 7)


size 열의 데이터 수를 확인해보면 1, 5, 6 테이블의 주문이 적다는 것을 알 수 있음

In [25]:
tips['size'].value_counts()

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64

만약 30번 이상의 주문이 있는 테이블만 추려 데이터 분석을 하려면 filter 메서드를 사용하여 해결할 수 있음

In [26]:
tips_filtered = tips.groupby('size').filter(lambda x:x['size'].count()>=30)
tips_filtered.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


1, 5, 6 테이블이 제외된 것을 확인할 수 있음.

In [27]:
tips_filtered['size'].value_counts()

2    156
3     38
4     37
Name: size, dtype: int64

# 그룹 오브젝트 저장하여 살펴보기
groupby 메서드가 반환하는 그룹 오브젝트에 대한 내용

In [28]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [29]:
grouped = tips_10.groupby('sex')
print(type(grouped))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


그룹 오브젝트에 포함된 그룹을 보려면 .groups 속성을 사용하면됨.  
그러면 sex 로 그룹화한 데이터프레임의 인덱스를 확인할 수 있음.  
오브젝트로 집계 변환, 필터 작업을 수행할 수 있음

In [30]:
grouped.groups

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}

# 그룹 오브젝트의 평균 구하기

앞선 grouped 오브젝트는 smoker, day, time 열과 같은 숫자형 데이터가 아니여서 평균값을 구할 수 없는 열도 포함하고 있음.  
하지만 오브젝트에 mean 과 같은 집계 메서드를 사용해도 에러가 나지 않음.  
파이썬은 자동으로 계산할 수 있는 열을 골라 계산을 수행하기 때문에 

In [31]:
# 숫자형 데이터인 total_bill, tips, size 만 계산된 것을 확인할 수 있음. 
grouped.mean()

,total_bill,tip,size
sex,,,
Male,20.02,2.875714,2.571429
Female,13.62,2.506667,2.000000


# 그룹 오브젝트에서 데이터 추출하고 반복하기
그룹 오브젝트에서 '데이터 추출하기'와 '반복문 사용하기'임.  

만일 그룹 오브젝트에서 특정 데이터만을 추출하려면 get_group 메서드를 사용하면 됨.  


In [32]:
# sex열로 그룹화한 오브젝트에서 sex==Female 인 데이터만 추출
female = grouped.get_group('Female')
female

,total_bill,tip,sex,smoker,day,time,size
198,13.00,2.00,Female,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


그룹 오브젝트를 반복문에 사용하여 활용하는 방법  
각 성별 그룹의 데이터를 반복문을 이용해서 출력하는 방법 

In [33]:
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


출력결과를 보면 tuple 값으로 리턴된 것을 확인할 수 있음.  

In [34]:
for sex_group in grouped:
    print('the type is : {}\n'.format(type(sex_group)))
    print('the length is : {}\n'.format(len(sex_group)))
    
    first_element = sex_group[0]
    print('the first element is : {}\n'.format(first_element))
    print('it has a type of : {}\n'.format(type(first_element)))
          
    second_element = sex_group[1]
    print('the second element is : {}\n'.format(second_element))
    print('it has a type of : {}\n'.format(type(second_element)))
          
    print('what we have : ')
    print(sex_group)
          
    break

the type is : <class 'tuple'>

the length is : 2

the first element is : Male

it has a type of : <class 'str'>

the second element is :      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2

it has a type of : <class 'pandas.core.frame.DataFrame'>

what we have : 
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192      

# 그룹 오브젝트 계산하고 살펴보기
여러 열을 사용하여 그룹 오브젝트를 만들고 평균값을 구하는 등의 계산도 가능

In [35]:
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean()
print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


group_avg 의 자료형을 확인해보면 데이터프레임임을 알 수 있음  
또한 변수 group_avg에 포함된 열은 total_bill, tip, size 라는 것도 알 수 있음.  

In [36]:
print(type(group_avg))
print(group_avg.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['total_bill', 'tip', 'size'], dtype='object')


group_avg 의 인덱스를 확인해보면 여러 인덱스를 가지고 있음  
이러한 인덱스는 특별히 Multiindex 라고 부름

In [37]:
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


데이터프레임의 인덱스가 multiindex 인 경우에는 reset_index() 메서드를 사용하여 데이터프레임의 인덱스를 새로 부여할 수 있음.  
>DataFrame.**reset_index**(level=None, drop=False, inplace=False, col_level=0, col_fill='')[source]  
>설명 : Reset the index, or a level of it.

In [38]:
group_method = tips_10.groupby(['sex', 'time']).mean().reset_index()
group_method

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000


reset_index 대신, groupby 의 as_index 의 인자값을 False로 설정해도 같은 결과를 얻을 수 있음

In [39]:
group_method = tips_10.groupby(['sex', 'time'], as_index=False).mean()
group_method

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000
